# Вспомогательные функции

In [1]:
# Вспомогательные функции
def distance(route, distances):
    
    distance = 0
    flag = 1
#     print(route)
    for dot in route: # [2, 1, 3]
        
        if flag == 0:
#             print("distances")
            distance += distances.iloc[prev_dot, dot] # prev_dot=2 , dot=1
            
        prev_dot = dot # 2
        flag = 0
    
    return distance

def check_points(points):
    
    for i, n in enumerate(points):
        if 0 in n:
            return i
    return None

def count_loading_unloading_nbr(start_b, points, start_g, loading_time=20, unloading_time=15):
    
    loading = 0
    unloading = 0
    
    for pair in points:
        if start_g in pair:
            uniq_count = set(pair) - {start_g}
            loading += (len(uniq_count) + list(uniq_count).count(3)) * loading_time
            unloading += (len(uniq_count) - list(uniq_count).count(3)) * unloading_time + (unloading_time + 5) * list(uniq_count).count(3)
        else:
            loading += len(pair) * loading_time
            unloading += (len(pair) - pair.count(3)) * unloading_time + pair.count(3) * (unloading_time + 5)
            
    return loading + unloading

# Введение

## Выгрузка данных

In [2]:
from datetime import time
import pandas as pd
import numpy as np
from itertools import permutations, chain
from copy import deepcopy

In [3]:
distances = pd.read_excel('временное расстояние объектов сети.xlsx', index_col = 0,  index = 0)
distances

,0,1,2,3,4,5,6,7
Объект,,,,,,,,
0,0,7,9,15,14,20,18,11
1,7,0,5,7,9,15,11,4
2,9,5,0,11,7,15,6,9
3,15,7,11,0,6,4,3,9
4,14,9,7,6,0,10,4,7
5,20,15,15,4,10,0,16,9
6,18,11,6,3,4,16,0,10
7,11,4,9,9,7,9,10,0


## Алгоритм для получения маршрута

### Алгоритм для получения маршрута для ПН, ВТ, ЧТ

In [4]:
def combinations_m_t_w(start, points, end, start_flag, start_g, distrib_flag):

    routes = [[start] * start_flag + [start_g] * distrib_flag + comb + [end] for comb in map(list, permutations(points))]
    return routes

def add_baking_route_m_t_w(start, points, start_g):
    
    final_route = []
    ind = check_points(points)
    
    for num, pair in enumerate(points):
        
        flag_end = 0 if start_g in pair or num + 1 == ind else 1
        distrib_flag = 1 if num == 0 else 0
        start_temp = start_g if start_g in pair else start
        start_ind = 1 if start_g in pair else 0
        start_flag = 0 if start_g in pair and len(pair) == 2 else 1
        end_temp = start_g if num + 1 == ind or num == len(points) - 1 else start
        possible_combinations = combinations_m_t_w(start_temp, pair[start_ind:], 
                                             end_temp, start_flag, start_g, distrib_flag)

        temp_dist = []

        for route in possible_combinations:
            temp_dist.append(distance(route, distances))
            
        min_dist = min(temp_dist)
            
        for i, route in enumerate(possible_combinations):

            if temp_dist[i] == min_dist:
                if num == 0:
                    route = route[1:]
                final_route.append(route if num == len(points) - 1 else route[:-1])
                break

    return list(chain(*final_route))


def baking_routes_m_t_w(start, pairs, start_g):
    
    final_routes = []
    final_dist = []

    for points in map(list, permutations(pairs)):
        loading_unloading_time = count_loading_unloading_nbr(start, points, start_g)
        final_route = add_baking_route_m_t_w(start, points, start_g)
        final_routes.append(final_route)
        final_dist.append(distance(final_route, distances))

    min_dist = min(final_dist)
    results = [route for i, route in enumerate(final_routes) if final_dist[i] == min_dist]
    distance_time = min_dist

    return results, loading_unloading_time, distance_time

def grocery_baking_routes_m_t_w(start_b, pairs_b, start_g, pairs_g):
    
    flag = 1
    final_pairs_g = []
    
    for pair in pairs_g:
        elems = []
        for elem in pair:
            elems.append([start_g] + elem)
        final_pairs_g.append(elems)
        
    pairs_b_copy = deepcopy(pairs_b)
    
    routes_g_b = []
    time_g_b = []

    for elems in final_pairs_g:
        for elem in elems:
            pairs_b_copy.append(elem)

        routes, loading_unloading, distance_time = baking_routes_m_t_w(start_b, pairs_b_copy, start_g)
        routes_g_b.append(routes)
        time_g_b.append(loading_unloading + distance_time)

        pairs_b_copy = deepcopy(pairs_b)

    min_time_g_b = min(time_g_b)
    the_best_routes = [routes for i, routes in enumerate(routes_g_b) if time_g_b[i] == min_time_g_b]
    
    return the_best_routes, min_time_g_b

In [5]:
start_b = 2
pairs_b = [[1, 3], [4, 5], [6, 7]]
start_g = 0
pairs_g = [[[1, 2, 3]]] #, [[1, 2], [3]]]

In [6]:
routes, minutes = grocery_baking_routes_m_t_w(start_b, pairs_b, start_g, pairs_g)

In [7]:
routes

[[[0, 1, 3, 2, 2, 1, 3, 2, 4, 5, 2, 6, 7, 0],
  [0, 1, 3, 2, 2, 4, 5, 2, 1, 3, 2, 6, 7, 0],
  [0, 1, 3, 2, 2, 4, 5, 2, 6, 7, 2, 3, 1, 0],
  [0, 1, 3, 2, 2, 6, 7, 2, 4, 5, 2, 3, 1, 0]]]

In [8]:
start_b2 = 2
pairs_b2 = [[1, 3], [4, 5], [6, 7]]
start_g2 = 0
pairs_g2 = [[[4, 5]]] #, [[1, 2], [3]]]

In [9]:
routes2, minutes2 = grocery_baking_routes_m_t_w(start_b2, pairs_b2, start_g2, pairs_g2)

In [10]:
routes2

[[[0, 5, 4, 2, 1, 3, 2, 4, 5, 2, 6, 7, 0],
  [0, 5, 4, 2, 4, 5, 2, 1, 3, 2, 6, 7, 0],
  [0, 5, 4, 2, 4, 5, 2, 6, 7, 2, 3, 1, 0],
  [0, 5, 4, 2, 6, 7, 2, 4, 5, 2, 3, 1, 0]]]

### Алгоритм получения маршрута для СБ

In [11]:
def combinations_s(start, points, end, start_flag, start_g, distrib_flag):

    routes = [[start_g] * distrib_flag + [start] * start_flag + comb + [end] for comb in map(list, permutations(points))]
    return routes

def add_baking_route_s(start, points, start_g):
    
    final_route = []
    ind = check_points(points)
    
    for num, pair in enumerate(points):

        flag_end = 1
        distrib_flag = 1 if num == 0 else 0
        start_temp = start_g if start_g in pair else start
        start_ind = 1 if start_g in pair else 0
        start_flag = 0 if start_g in pair and len(pair) == 2 else 1
        end_temp = start_g if num + 1 == ind or num == len(points) - 1 else start
        possible_combinations = combinations_s(start_temp, pair[start_ind:], end_temp, start_flag, 
                                             start_g, distrib_flag)

        temp_dist = []

        for route in possible_combinations:
            temp_dist.append(distance(route, distances))
            
        min_dist = min(temp_dist)
            
        for i, route in enumerate(possible_combinations):

            if temp_dist[i] == min_dist:
                final_route.append(route if num == len(points) - 1 else route[:-1]) 
                break

    return list(chain(*final_route))

def baking_routes_s(start, pairs, start_g):
    
    final_routes = []
    final_dist = []

    for points in map(list, permutations(pairs)):
        loading_unloading_time = count_loading_unloading_nbr(start, points, start_g)
        final_route = add_baking_route_s(start, points, start_g)
        final_routes.append(final_route)
        final_dist.append(distance(final_route, distances))

    min_dist = min(final_dist)
    results = [route for i, route in enumerate(final_routes) if final_dist[i] == min_dist]
    distance_time = min_dist

    return results, loading_unloading_time, distance_time

In [12]:
baking_routes_s(start_b, pairs_b, start_g)

([[0, 2, 1, 3, 2, 4, 5, 2, 6, 7, 0],
  [0, 2, 4, 5, 2, 1, 3, 2, 6, 7, 0],
  [0, 2, 4, 5, 2, 6, 7, 2, 3, 1, 0],
  [0, 2, 6, 7, 2, 4, 5, 2, 3, 1, 0]],
 215,
 91)

### Алгоритм получения маршрута для СР и ПТ
### 2-х тонник

In [13]:
def check_points_w_f(points):
    
    for i, n in enumerate(points):
        if n[0] == 3:
            return i
    return None

def combinations_w_f(start, points, end, start_flag, start_g, distrib_flag):

    routes = [[start_g] * distrib_flag + [start] * start_flag + comb + [end] for comb in map(list, permutations(points))]
    return routes

def add_baking_route_w_f(start, points, start_g, start_c):
    
    final_route = []
    ind = check_points_w_f(points)
    
    for num, pair in enumerate(points):

        flag_end = 1
        distrib_flag = 1 if num == 0 else 0
        start_temp = start_g if start_g in pair else start_c if pair[0] == start_c else start
        start_ind = 1 if start_g in pair or pair[0] == start_c else 0
        start_flag = 0 if start_g in pair else 1
        
        if num + 1 == ind and num != len(points) - 1:
            end_temp = start_c
        elif num == len(points) - 1:
            end_temp = start_g
        else:
            end_temp = start
            
        possible_combinations = combinations_w_f(start_temp, pair[start_ind:], end_temp, start_flag, 
                                             start_g, distrib_flag)

        temp_dist = []

        for route in possible_combinations:
            temp_dist.append(distance(route, distances))
            
        min_dist = min(temp_dist)
            
        for i, route in enumerate(possible_combinations):

            if temp_dist[i] == min_dist:
                final_route.append(route if num == len(points) - 1 else route[:-1]) 
                break

    return list(chain(*final_route))

def baking_routes_w_f(start, pairs, start_g, start_c):
    
    final_routes = []
    final_dist = []

    for points in map(list, permutations(pairs)):

        loading_unloading_time = 0
        final_route = add_baking_route_w_f(start, points, start_g, start_c)
        final_routes.append(final_route)
        final_dist.append(distance(final_route, distances))

    min_dist = min(final_dist)
    results = [route for i, route in enumerate(final_routes) if final_dist[i] == min_dist]
    distance_time = min_dist

    return results, loading_unloading_time, distance_time


def chocolate_baking_routes_w_f(start_b, pairs_b, start_g, start_c, pairs_c):
    
    elems = []
    for pair in pairs_c:
        elems.append([start_c] + pair)
        
    pairs_b_copy = deepcopy(pairs_b)
    
    routes_c_b = []
    time_c_b = []
        
    for elem in elems:
        pairs_b_copy.append(elem)
        
        routes, loading_unloading, distance_time = baking_routes_w_f(start_b, pairs_b_copy, start_g, start_c)
        routes_c_b.append(routes)
        time_c_b.append(loading_unloading + distance_time)

        pairs_b_copy = deepcopy(pairs_b)
    
    min_time_c_b = min(time_c_b)
    the_best_routes = [routes for i, routes in enumerate(routes_c_b) if time_c_b[i] == min_time_c_b]
    
    return the_best_routes, min_time_c_b

In [14]:
start_b3 = 2
pairs_b3 = [[1, 3], [4, 5], [6, 7]]
start_g3 = 0
start_c = 3
pairs_c = [[1, 2], [4, 5], [6, 7]]

chocolate_baking_routes_w_f(start_b3, pairs_b3, start_g3, start_c, pairs_c)

([[[0, 2, 1, 3, 2, 4, 5, 3, 1, 2, 2, 6, 7, 0],
   [0, 2, 1, 3, 3, 1, 2, 2, 4, 5, 2, 6, 7, 0],
   [0, 2, 4, 5, 2, 1, 3, 3, 1, 2, 2, 6, 7, 0],
   [0, 2, 4, 5, 3, 1, 2, 2, 1, 3, 2, 6, 7, 0],
   [0, 2, 4, 5, 3, 1, 2, 2, 6, 7, 2, 3, 1, 0],
   [0, 2, 6, 7, 2, 4, 5, 3, 1, 2, 2, 3, 1, 0]]],
 92)

### 1.5-ра тонник

In [15]:
def check_points_chocolate_w_f(points):
    
    for i, n in enumerate(points):
        if n[0] == 3:
            return i
    return None

def combinations_choc_w_f(start, points, end, start_flag, start_g, distrib_flag):

    routes = [[start_g] * distrib_flag + [start] * start_flag + comb + [end] for comb in map(list, permutations(points))]
    return routes

def add_grocery_route_choc_w_f(start_g, points, start_c):
    
    final_route = []
    ind = check_points_chocolate_w_f(points)
    for num, pair in enumerate(points):

        flag_end = 1
        distrib_flag = 1 if num == 0 else 0
        start_temp = start_c if pair[0] == start_c else start_g 
        start_ind = 1 if pair[0] == start_c else 0
        start_flag = 0 if pair[0] != start_c else 1
        
        if num + 1 == ind and num != len(points) - 1:
            end_temp = start_c
        elif num == len(points) - 1:
            end_temp = start_g
        else:
            end_temp = start_g
            
        possible_combinations = combinations_choc_w_f(start_temp, pair[start_ind:], end_temp, start_flag, 
                                             start_g, distrib_flag)

        temp_dist = []

        for route in possible_combinations:
            temp_dist.append(distance(route, distances))
            
        min_dist = min(temp_dist)
            
        for i, route in enumerate(possible_combinations):

            if temp_dist[i] == min_dist:
                final_route.append(route) 
                break

    return list(chain(*final_route))

def grocery_routes_choc_w_f(start_g, pairs_g, start_c):
    
    final_routes = []
    final_dist = []

    for points in map(list, permutations(pairs_g)):
#         loading_unloading_time = count_loading_unloading_nbr(start, points, start_g)
        loading_unloading_time = 0
        final_route = add_grocery_route_choc_w_f(start_g, pairs_g, start_c)
        final_routes.append(final_route)
        final_dist.append(distance(final_route, distances))

    min_dist = min(final_dist)
    results = [route for i, route in enumerate(final_routes) if final_dist[i] == min_dist]
    distance_time = min_dist

    return results, loading_unloading_time, distance_time

def chocolate_grocery_routes_choc_w_f(start_g, pairs_g, start_c, pairs_c):
    
    final_pairs_c = []
    for pair in pairs_c:
        elems = []
        for elem in pair:
            elems.append([start_c] + elem)
        final_pairs_c.append(elems)
        
    pairs_g_copy = deepcopy(pairs_g)
    routes_c_g = []
    time_c_g = []
        
    for elems in final_pairs_c:
        for elem in elems:
            pairs_g_copy.append(elem)
        
        routes, loading_unloading, distance_time = grocery_routes_choc_w_f(start_g, pairs_g_copy, start_c)
        routes_c_g.append(routes)
        time_c_g.append(loading_unloading + distance_time)

        pairs_g_copy = deepcopy(pairs_g)
    
    min_time_c_g = min(time_c_g)
    the_best_routes = [routes for i, routes in enumerate(routes_c_g) if time_c_g[i] == min_time_c_g]
    
    return the_best_routes[0][0], min_time_c_g

In [16]:
start_g3 = 0
pairs_g3 = [[1, 2, 3], [6, 7]]
start_c = 3
pairs_c = [[[6]]]

chocolate_grocery_routes_choc_w_f(start_g3, pairs_g3, start_c, pairs_c)

([0, 1, 3, 2, 0, 7, 6, 3, 3, 6, 0], 79)

In [17]:
start_g3 = 0
pairs_g3 = [[1, 2, 3], [6, 7]]
start_c = 3
pairs_c = [[[7]]]

chocolate_grocery_routes_choc_w_f(start_g3, pairs_g3, start_c, pairs_c)

([0, 1, 3, 2, 0, 7, 6, 3, 3, 7, 0], 78)

## Алгоритм для получения расписания

In [18]:
def get_schedual_monday(routes, day, capacity): 
    
    dots_names = {
        0: "КЦ",
        1: "Кафе 1",
        2: "Кафе 2",
        3: "Кафе 3",
        4: "Супермаркет 1",
        5: "Супермаркет 2",
        6: "Супермаркет 3",
        7: "Супермаркет 4"
    }

    cargo_type = {2: ['выпечка', 0.8], 0: ['бакалея', 0.5]}
    num_dots = 3

    dispatch_time = 10 * 60
    route = routes
    cargo = cargo_type[0]
    
    print(f"РАСПИСАНИЕ ДЛЯ {day}")
    print(f"РАСПИСАНИЕ ДЛЯ {capacity}-ТОННИКА")
    print()
    print("Точка               Время             Время      Доставленно   Доставленно   Взято      Взято")
    print("                  прибытия         отправления    груза,т       груза,тип    груза,т    груза,тип")
    print()

    for i, dot in enumerate(route):
        
        loading_flag = 1 if dot == 0 or dot == 2 and route[i + 1] != 2 else 0
        unloading_flag = 0 if loading_flag else 1
        if dot == 2:
            num_dots = 2
        additional_flag = 5 if dot == 3 else 0
        additional_delivery = 15 if unloading_flag and dot == 1 and route[i - 1] == 2 else 20 if loading_flag and dot == 2 and route[i + 1] == 1 else 0
        arrival_time = dispatch_time + distance([route[i - 1], route[i]], distances)
        dispatch_time = arrival_time + 20 * loading_flag * num_dots + 15 * unloading_flag + additional_flag + additional_delivery if i != len(route) -1 else '  -  '
        if dot == 2 and route[i + 1] == 2:
            arrival_first = arrival_time
            continue

        if dot == 2 and route[i - 1] == 2:
            
            print(f'{dots_names[dot]}' , end = '')
            print(' ' * (20 - len(dots_names[dot])), end = '')
            print(f'{arrival_first // 60}:{(arrival_first % 60):02d}', end='')
            print(' ' * 12, end='')
            print(f'{dispatch_time // 60}:{(dispatch_time % 60):02d}', end='')

            if dot == 0 or (dot == 2 and route[i + 1] == 2):
                cargo = cargo_type[0]
            if dot == 2 and route[i - 1] == 2:
                   cargo = cargo_type[2]
            cargo_unloaded = 0.5
            cargo_type_unloaded = 'бакалея'
        else:   
            print(f'{dots_names[dot]}' , end = '')
            print(' ' * (20 - len(dots_names[dot])), end = '')
            print(f'{arrival_time // 60}:{(arrival_time % 60):02d}', end='')
            print(' ' * 12, end = '')
            if dispatch_time != '  -  ':
                print(f'{dispatch_time // 60}:{(dispatch_time % 60):02d}', end='')
            else:
                print(f'{dispatch_time}', end='')

            if dot == 0 or (dot == 2 and route[i + 1] == 2):
                cargo = cargo_type[0]
            if dot == 2:
                cargo = cargo_type[2]

            cargo_unloaded = cargo[1] if (dot != 2 or (dot == 2 and route[i + 1] == 2)) and dot != 0 else ' - '
            if cargo_unloaded != ' - ' and dot == 1 and route[i - 1] == 2:
                cargo_unloaded += 0.1
                
            cargo_type_unloaded = cargo[0] if (dot != 2 or (dot == 2 and route[i + 1] == 2)) and dot != 0 else '   -   '
            if cargo_type_unloaded != ' - ' and dot == 1 and route[i - 1] == 2:
                cargo_type_unloaded = 'вып+доз'
            
        cargo_loaded = cargo[1] * num_dots if (dot == 2 and route[:i + 1].count(2) > 1) or (dot == 0 and i != len(route) -1) else ' - '
        if cargo_loaded != ' - ' and dot == 2 and route[i + 1] == 1:
            cargo_loaded += 0.1
            cargo_loaded = round(cargo_loaded, 1)
            
        cargo_type_loaded = cargo[0] if (dot == 2 and route[:i + 1].count(2) > 1) or (dot == 0 and i != len(route) -1) else '   -   '
        if cargo_type_loaded != ' - ' and dot == 2 and route[i + 1] == 1:
            cargo_type_loaded = 'вып+доз'

        print(' ' * 5, end = '')
        print(f'    {cargo_unloaded}', end='')
        print(' ' * (10 - len(str(cargo_unloaded))), end = '')
        print(f'    {cargo_type_unloaded}', end = '')
        print(' ' * 3, end = '')
        print(f'    {cargo_loaded}', end='')
        print(' ' * (7 - len(str(cargo_loaded))), end = '')
        print(f'    {cargo_type_loaded}')

In [19]:
def get_schedual_tuesday(routes, day, capacity): 
    
    dots_names = {
        0: "КЦ",
        1: "Кафе 1",
        2: "Кафе 2",
        3: "Кафе 3",
        4: "Супермаркет 1",
        5: "Супермаркет 2",
        6: "Супермаркет 3",
        7: "Супермаркет 4"
    }

    cargo_type = {2: ['выпечка', 0.8], 0: ['бакалея', 0.5]}

    dispatch_time = 10 * 60
    route = routes
    cargo = cargo_type[0]
    
    print(f"РАСПИСАНИЕ ДЛЯ {day}")
    print(f"РАСПИСАНИЕ ДЛЯ {capacity}-ТОННИКА")
    print()
    print("Точка               Время             Время      Доставленно   Доставленно   Взято      Взято")
    print("                  прибытия         отправления    груза,т       груза,тип    груза,т    груза,тип")
    print()

    for i, dot in enumerate(route):
        
        loading_flag = 1 if dot == 0 or dot == 2 and route[i + 1] != 2 else 0
        unloading_flag = 0 if loading_flag else 1
        num_dots = 2
        additional_flag = 5 if dot == 3 else 0
        arrival_time = dispatch_time + distance([route[i - 1], route[i]], distances)
        additional_delivery = 15 if unloading_flag and dot == 1 and route[i - 1] == 2 else 20 if loading_flag and dot == 2 and route[i + 1] == 1 else 0
        dispatch_time = arrival_time + 20 * loading_flag * num_dots + 15 * unloading_flag + additional_flag + additional_delivery if i != len(route) -1 else '  -  '
        
        print(f'{dots_names[dot]}' , end = '')
        print(' ' * (20 - len(dots_names[dot])), end = '')
        print(f'{arrival_time // 60}:{(arrival_time % 60):02d}', end='')
        print(' ' * 12, end = '')
        
        if dispatch_time != '  -  ':
            print(f'{dispatch_time // 60}:{(dispatch_time % 60):02d}', end='')
        else:
            print(f'{dispatch_time}', end='')

        if dot == 0 or (dot == 2 and route[i + 1] == 2):
            cargo = cargo_type[0]
        if dot == 2:
            cargo = cargo_type[2]

        cargo_unloaded = cargo[1] if dot != 2  and dot != 0 else ' - '
        if cargo_unloaded != ' - ' and dot == 1 and route[i - 1] == 2:
            cargo_unloaded += 0.1
        cargo_type_unloaded = cargo[0] if dot != 2 and dot != 0 else '   -   '
        if cargo_type_unloaded != ' - ' and dot == 1 and route[i - 1] == 2:
            cargo_type_unloaded = 'вып+доз'
    
        cargo_loaded = cargo[1] * num_dots if dot == 2 or (dot == 0 and i != len(route) -1) else ' - '
        if cargo_loaded != ' - ' and dot == 2 and route[i + 1] == 1:
            cargo_loaded += 0.1
            cargo_loaded = round(cargo_loaded, 1)
        cargo_type_loaded = cargo[0] if dot == 2  or (dot == 0 and i != len(route) -1) else '   -   '
        if cargo_type_loaded != ' - ' and dot == 2 and route[i + 1] == 1:
            cargo_type_loaded = 'вып+доз'
            
        print(' ' * 5, end = '')
        print(f'    {cargo_unloaded}', end='')
        print(' ' * (10 - len(str(cargo_unloaded))), end = '')
        print(f'    {cargo_type_unloaded}', end = '')
        print(' ' * 3, end = '')
        print(f'    {cargo_loaded}', end='')
        print(' ' * (7 - len(str(cargo_loaded))), end = '')
        print(f'    {cargo_type_loaded}')

In [20]:
def get_schedual_w_f(routes, day, capacity): 
    
    dots_names = {
        0: "КЦ",
        1: "Кафе 1",
        2: "Кафе 2",
        3: "Кафе 3",
        4: "Супермаркет 1",
        5: "Супермаркет 2",
        6: "Супермаркет 3",
        7: "Супермаркет 4"
    }

    cargo_type = {2: ['выпечка', 0.8], 0: ['бакалея', 0.5], 3: ['шоколад', 0.8]}

    dispatch_time = 10 * 60
    route = routes
    cargo = cargo_type[3]
    
    print(f"РАСПИСАНИЕ ДЛЯ {day}")
    print(f"РАСПИСАНИЕ ДЛЯ {capacity}-ТОННИКА")
    print()
    print("Точка               Время             Время      Доставленно   Доставленно   Взято      Взято")
    print("                  прибытия         отправления    груза,т       груза,тип    груза,т    груза,тип")
    print()
 
    if capacity == 2:
        
        for i, dot in enumerate(route):

            loading_flag = 1 if (dot == 3 and route[i + 1] != 3) or (dot == 2 and route[i + 1] != 2) else 0
            unloading_flag = 0 if loading_flag else 1
            num_dots = 2
            additional_flag = 5 * unloading_flag + (5 * num_dots * loading_flag) if dot == 3 else 0
            additional_delivery = 15 if unloading_flag and dot == 1 and route[i - 1] == 2 else 20 if loading_flag and dot == 2 and route[i + 1] == 1 else 0
            arrival_time = dispatch_time + distance([route[i - 1], route[i]], distances) if i != 0 else '  -  '
            
            if arrival_time == '  -  ':
                dispatch_time = dispatch_time
            else:
                dispatch_time = arrival_time + 20 * loading_flag * num_dots + 15 * unloading_flag + additional_flag + additional_delivery if i != len(route) - 1 else '  -  '

            if dot == 3 and route[i + 1] == 3:
                arrival_first = arrival_time
                continue
                
                
            if dot == 3 and route[i - 1] == 3:
                
                print(f'{dots_names[dot]}' , end = '')
                print(' ' * (20 - len(dots_names[dot])), end = '')
                print(f'{arrival_first // 60}:{(arrival_first % 60):02d}', end='')
                print(' ' * 12, end='')
                print(f'{dispatch_time // 60}:{(dispatch_time % 60):02d}', end='')

                cargo_unloaded = 0.8
                cargo_type_unloaded = 'выпечка'

                cargo_loaded = 1.6
                cargo_type_loaded = 'шоколад'
                
            elif dot == 2 and route[i + 1] == 2:
                arrival_first = arrival_time
                continue

            elif dot == 2 and route[i - 1] == 2:

                print(f'{dots_names[dot]}' , end = '')
                print(' ' * (20 - len(dots_names[dot])), end = '')
                print(f'{arrival_first // 60}:{(arrival_first % 60):02d}', end='')
                print(' ' * 12, end='')
                print(f'{dispatch_time // 60}:{(dispatch_time % 60):02d}', end='')

                cargo_unloaded = 0.8
                cargo_type_unloaded = 'шоколад'

                cargo_loaded = 1.6
                cargo_type_loaded = 'выпечка'
            else:
                
                print(f'{dots_names[dot]}' , end = '')
                print(' ' * (20 - len(dots_names[dot])), end = '')

                if arrival_time != '  -  ':
                    print(f'{arrival_time // 60}:{(arrival_time % 60):02d}', end='')
                else:
                    print(f'{arrival_time}', end='')

                print(' ' * 12, end = '')
                if dispatch_time != '  -  ':
                    print(f'{dispatch_time // 60}:{(dispatch_time % 60):02d}', end='')
                else:
                    print(f'{dispatch_time}', end='')

                if dot == 2 or route[i - 1] == 2:
                    cargo = cargo_type[2]
                elif dot == 3 or route[i - 1] == 3:
                    cargo = cargo_type[3]

                cargo_unloaded = cargo[1] if dot != 2 and (dot != 3 or route[:i + 1].count(3))  <= 1 and dot != 0 else ' - '
                if cargo_unloaded != ' - ' and dot == 1 and route[i - 1] == 2:
                    cargo_unloaded += 0.1
                    
                cargo_type_unloaded = cargo[0] if dot != 2 and (dot != 3 or route[:i + 1].count(3) <= 1) and dot != 0 else '   -   '
                if cargo_type_unloaded != ' - ' and dot == 1 and route[i - 1] == 2:
                    cargo_type_unloaded = 'вып+доз'
                    
                cargo_loaded = cargo[1] * num_dots if (dot == 2 or dot == 3) and dot != 0 else ' - '
                if cargo_loaded != ' - ' and dot == 2 and route[i + 1] == 1:
                    cargo_loaded += 0.1
                    cargo_loaded = round(cargo_loaded, 1)
                    
                cargo_type_loaded = cargo[0] if (dot == 2 or dot == 3) and dot != 0 else '   -   '
                if cargo_type_loaded != ' - ' and dot == 2 and route[i + 1] == 1:
                    cargo_type_loaded = 'вып+доз'

            print(' ' * 5, end = '')
            print(f'    {cargo_unloaded}', end='')
            print(' ' * (10 - len(str(cargo_unloaded))), end = '')
            print(f'    {cargo_type_unloaded}', end = '')
            print(' ' * 3, end = '')
            print(f'    {cargo_loaded}', end='')
            print(' ' * (7 - len(str(cargo_loaded))), end = '')
            print(f'    {cargo_type_loaded}')
            
    if capacity == 1.5:
        
        num_dots = 3
        
        for i, dot in enumerate(route):

            loading_flag = 1 if dot == 0 or (dot == 3 and route[i - 1] == 3) else 0
            unloading_flag = 0 if loading_flag or dot == 3 and route[i + 1] == 3 else 1
            
            if dot == 0 and route[:i + 1].count(0) > 1:
                num_dots = 2
            if dot == 3 and route[i - 1] == 3:
                num_dots = 1
                
            additional_flag = 5 if dot == 3 and (loading_flag or unloading_flag) else 0
            arrival_time = dispatch_time + distance([route[i - 1], route[i]], distances)
            dispatch_time = arrival_time + 20 * loading_flag * num_dots + 15 * unloading_flag + additional_flag if i != len(route) - 1 else '  -  '

            if dot == 3 and route[i + 1] == 3:
                arrival_first = arrival_time
                continue
                
                
            if dot == 3 and route[i - 1] == 3:
                
                print(f'{dots_names[dot]}' , end = '')
                print(' ' * (20 - len(dots_names[dot])), end = '')
                print(f'{arrival_first // 60}:{(arrival_first % 60):02d}', end='')
                print(' ' * 12, end='')
                print(f'{dispatch_time // 60}:{(dispatch_time % 60):02d}', end='')
                
            else:
                
                print(f'{dots_names[dot]}' , end = '')
                print(' ' * (20 - len(dots_names[dot])), end = '')
                print(f'{arrival_time // 60}:{(arrival_time % 60):02d}', end='')

                print(' ' * 12, end = '')
                if dispatch_time != '  -  ':
                    print(f'{dispatch_time // 60}:{(dispatch_time % 60):02d}', end='')
                else:
                    print(f'{dispatch_time}', end='')

            if dot == 0:
                cargo = cargo_type[0]
            if dot == 3 and route[i - 1] == 3:
                cargo = cargo_type[3]

            cargo_unloaded = cargo[1] if (dot != 3 or (dot == 3 and route[i - 1] != 3)) and dot != 0 else ' - '
            cargo_type_unloaded = cargo[0] if (dot != 3 or (dot == 3 and route[i - 1] != 3)) and dot != 0 else '   -   '

            cargo_loaded = cargo[1] * num_dots if (dot == 3 and route[:i + 1].count(3) > 1) or (dot == 0 and i != len(route) -1) else ' - '
            cargo_type_loaded = cargo[0] if (dot == 3 and route[:i + 1].count(3) > 1) or (dot == 0 and i != len(route) -1) else '   -   '

            print(' ' * 5, end = '')
            print(f'    {cargo_unloaded}', end='')
            print(' ' * (10 - len(str(cargo_unloaded))), end = '')
            print(f'    {cargo_type_unloaded}', end = '')
            print(' ' * 3, end = '')
            print(f'    {cargo_loaded}', end='')
            print(' ' * (7 - len(str(cargo_loaded))), end = '')
            print(f'    {cargo_type_loaded}')

In [21]:
def get_schedual_saturday(routes, day, capacity): 
    
    dots_names = {
        0: "КЦ",
        1: "Кафе 1",
        2: "Кафе 2",
        3: "Кафе 3",
        4: "Супермаркет 1",
        5: "Супермаркет 2",
        6: "Супермаркет 3",
        7: "Супермаркет 4"
    }

    cargo_type = {2: ['выпечка', 0.8]}

    dispatch_time = 10 * 60
    route = routes
    cargo = cargo_type[2]
    
    print(f"РАСПИСАНИЕ ДЛЯ {day}")
    print(f"РАСПИСАНИЕ ДЛЯ {capacity}-ТОННИКА")
    print()
    print("Точка               Время             Время      Доставленно   Доставленно   Взято      Взято")
    print("                  прибытия         отправления    груза,т       груза,тип    груза,т    груза,тип")
    print()

    for i, dot in enumerate(route):
        
        loading_flag = 1 if dot == 0 or dot == 2 and route[i + 1] != 2 else 0
        unloading_flag = 0 if loading_flag else 1
        num_dots = 2
        additional_flag = 5 if dot == 3 else 0
        arrival_time = dispatch_time + distance([route[i - 1], route[i]], distances) if i != 0 else '  -  '
        additional_delivery = 15 if unloading_flag and dot == 1 and route[i - 1] == 2 else 20 if loading_flag and dot == 2 and route[i + 1] == 1 else 0
        
        if arrival_time == '  -  ':
            dispatch_time = dispatch_time
        else:
            dispatch_time = arrival_time + 20 * loading_flag * num_dots + 15 * unloading_flag + additional_flag + additional_delivery if i != len(route) - 1 else '  -  '
        
        
        print(f'{dots_names[dot]}' , end = '')
        print(' ' * (20 - len(dots_names[dot])), end = '')
        if arrival_time != '  -  ':
            print(f'{arrival_time // 60}:{(arrival_time % 60):02d}', end='')
        else:
            print(f'{arrival_time}', end='')
        print(' ' * 12, end = '')
        
        if dispatch_time != '  -  ':
            print(f'{dispatch_time // 60}:{(dispatch_time % 60):02d}', end='')
        else:
            print(f'{dispatch_time}', end='')


        cargo_unloaded = cargo[1] if dot != 2  and dot != 0 else ' - '
        if cargo_unloaded != ' - ' and dot == 1 and route[i - 1] == 2:
            cargo_unloaded += 0.1
            
        cargo_type_unloaded = cargo[0] if dot != 2 and dot != 0 else '   -   '
        if cargo_type_unloaded != ' - ' and dot == 1 and route[i - 1] == 2:
            cargo_type_unloaded = 'вып+доз'
    
        cargo_loaded = cargo[1] * num_dots if dot == 2 else ' - '
        if cargo_loaded != ' - ' and dot == 2 and route[i + 1] == 1:
            cargo_loaded += 0.1
            cargo_loaded = round(cargo_loaded, 1)
            
        cargo_type_loaded = cargo[0] if dot == 2 else '   -   '
        if cargo_type_loaded != ' - ' and dot == 2 and route[i + 1] == 1:
            cargo_type_loaded = 'вып+доз'
            
        print(' ' * 5, end = '')
        print(f'    {cargo_unloaded}', end='')
        print(' ' * (10 - len(str(cargo_unloaded))), end = '')
        print(f'    {cargo_type_unloaded}', end = '')
        print(' ' * 3, end = '')
        print(f'    {cargo_loaded}', end='')
        print(' ' * (7 - len(str(cargo_loaded))), end = '')
        print(f'    {cargo_type_loaded}')

## Итоговое расписание

In [22]:
start_b = 2
start_g = 0
start_c = 3
pairs_b = [[1, 3], [4, 5], [6, 7]]
total_routes2 = []
total_routes1_5 = []

weekdays = ['ПОНЕДЕЛЬНИКА', 'ВТОРНИКА', 'СРЕДЫ', 'ЧЕТВЕРГА', 'ПЯТНИЦЫ', 'СУББОТЫ']

for weekday in weekdays:
    
    if weekday == 'ПОНЕДЕЛЬНИКА':
        start_b = 2
        pairs_b1 = [[1, 4], [5, 3], [6, 7]]
        start_g = 0
        pairs_g = [[[1, 2, 3]]]
        routes_m, minutes = grocery_baking_routes_m_t_w(start_b, pairs_b1, start_g, pairs_g)
       
        get_schedual_monday(routes_m[0][0], weekday, 2)
        total_routes2.append(routes_m[0][0])
    print()
    
    if weekday == 'ВТОРНИКА':
        pairs_b2 = [[1, 4], [5, 3], [6, 7]]
        pairs_g = [[[4, 5]]]
        routes_t, minutes = grocery_baking_routes_m_t_w(start_b, pairs_b2, start_g, pairs_g)
        get_schedual_tuesday(routes_t[0][0], weekday, 2)
        total_routes2.append(routes_t[0][0])
    print()
    
    if weekday == 'СРЕДЫ':
        start_b3 = 2
        pairs_b3 = [[1, 4], [5, 3], [6, 7]]
        start_g3 = 0
        start_c = 3
        pairs_c = [[1, 7]]

        routes_w, minutes = chocolate_baking_routes_w_f(start_b3, pairs_b3, start_g3, start_c, pairs_c)
        get_schedual_w_f(routes_w[0][0], weekday, 2)
        total_routes2.append(routes_w[0][0])
        print()
        
        start_g3 = 0
        pairs_g3 = [[1, 3, 7], [6, 2]]
        start_c = 3
        pairs_c = [[[4]]]

        routes_w2, minutes = chocolate_grocery_routes_choc_w_f(start_g3, pairs_g3, start_c, pairs_c)
        get_schedual_w_f(routes_w2, weekday, 1.5)
        total_routes1_5.append(routes_w2)
    print()
    
    if weekday == 'ЧЕТВЕРГА':
        pairs_b4 = [[1, 4], [5, 3], [6, 7]]
        pairs_g = [[[4, 5]]]
        routes_th, minutes = grocery_baking_routes_m_t_w(start_b, pairs_b4, start_g, pairs_g)
        get_schedual_tuesday(routes_th[0][0], weekday, 2)
        total_routes2.append(routes_th[0][0])
    print()
    
    if weekday == 'ПЯТНИЦЫ':
        start_b3 = 2
        pairs_b3 = [[1, 3], [4, 5], [6, 7]]
        start_g3 = 0
        start_c = 3
        pairs_c = [[2, 6]]

        routes_f, minutes = chocolate_baking_routes_w_f(start_b3, pairs_b3, start_g3, start_c, pairs_c)
        get_schedual_w_f(routes_f[0][1], weekday, 2)
        total_routes2.append(routes_f[0][1])
        print()
        
        start_g3 = 0
        pairs_g3 = [[1, 7, 3], [6, 2]]
        start_c = 3
        pairs_c = [[[5]]]

        routes_w2, minutes = chocolate_grocery_routes_choc_w_f(start_g3, pairs_g3, start_c, pairs_c)
        get_schedual_w_f(routes_w2, weekday, 1.5)
        total_routes1_5.append(routes_w2)
    print()
    
    if weekday == 'СУББОТЫ':
        pairs_b5 = [[1, 4], [5, 3], [6, 7]]
        routes_s = baking_routes_s(start_b, pairs_b5, start_g)
        get_schedual_saturday(routes_s[0][0], weekday, 2)
        total_routes2.append(routes_s[0][0])

РАСПИСАНИЕ ДЛЯ ПОНЕДЕЛЬНИКА
РАСПИСАНИЕ ДЛЯ 2-ТОННИКА

Точка               Время             Время      Доставленно   Доставленно   Взято      Взято
                  прибытия         отправления    груза,т       груза,тип    груза,т    груза,тип

КЦ                  10:00            11:00          -               -          1.5        бакалея
Кафе 1              11:07            11:22         0.5           бакалея        -            -   
Кафе 3              11:29            11:49         0.5           бакалея        -            -   
Кафе 2              12:00            13:15         0.5           бакалея       1.7        вып+доз
Кафе 1              13:20            13:50         0.9           вып+доз        -            -   
Супермаркет 1       13:59            14:14         0.8           выпечка        -            -   
Кафе 2              14:21            15:01          -               -          1.6        выпечка
Супермаркет 2       15:16            15:31         0.8           вы

## Расчет итоговых показателей

In [23]:
total_time2 = [distance(route, distances) for route in total_routes2]
total_time1_5 = [distance(route, distances) for route in total_routes1_5]

In [37]:
cost_result = sum(total_time2) * 4 * 30 / 60 * 7.2 / 50 * 1.39 + 400 + 900 \
                        + sum(total_time1_5) * 4 * 30 / 60 * 5.8 / 50 * 1.39 + 300 + 900

In [38]:
print(f'ИТОГОВЫЕ ЗАТРАТЫ {cost_result}€')

ИТОГОВЫЕ ЗАТРАТЫ 2789.12€


In [39]:
cost_result_opt = sum(total_time2) * 4 * 30 / 60 * 7.2 / 50 * 1.39 + 400 + 900 \
                        + sum(total_time1_5) * 4 * 30 / 60 * 5.8 / 50 * 1.39 + 300 + (900 / 24 * 16)

In [40]:
print(f'ИТОГОВЫЕ ЗАТРАТЫ C ОПТИМИЗАЦИЕЙ ЗП ПО ДНЯМ {cost_result_opt}€')

ИТОГОВЫЕ ЗАТРАТЫ C ОПТИМИЗАЦИЕЙ ЗП ПО ДНЯМ 2489.12€


In [41]:
cost_result_hours = sum(total_time2) * 4 * 30 / 60 * 7.2 / 50 * 1.39 + 400 + 710.45 \
                        + sum(total_time1_5) * 4 * 30 / 60 * 5.8 / 50 * 1.39 + 300 + 178.18

In [42]:
print(f'ИТОГОВЫЕ ЗАТРАТЫ C ОПТИМИЗАЦИЕЙ ЗП ПО ЧАСАМ {cost_result_hours}€')

ИТОГОВЫЕ ЗАТРАТЫ C ОПТИМИЗАЦИЕЙ ЗП ПО ЧАСАМ 1877.75€
